In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Elimina todas las variables meteorológicas que no correspondan a la localización de Sotavento (la localización 13)
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Elimina las variables 'lai_hv.13' y 'v10n.13'
if 'lai_hv.13' in wind_ava.columns and 'v10n.13' in wind_ava.columns:
    wind_ava = wind_ava.drop(['lai_hv.13', 'v10n.13'], axis=1)

# Vuelve a definir las variables numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Dividir los datos en conjuntos de entrenamiento y prueba (evaluación externa)
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Transforma los datos de entrenamiento y de competición con el método de escalado
X_train_scaled = MinMaxScaler.transform(X_train[numerical_vars])

# Apartado 6

In [ ]:
import pickle
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.ensemble import RandomForestClassifier

# Carga el modelo final
with open('modelo_final.pkl', 'rb') as file:
    best_model = pickle.load(file)

# Hace predicciones sobre los datos de entrenamiento
predictions = best_model.predict(X_train_scaled)

# Calcula el error para valores altos y bajos de energía
third_quantile = np.quantile(y_train, 0.75)
low_values_error = mean_squared_error(y_train[y_train < third_quantile], predictions[y_train < third_quantile])
high_values_error = mean_squared_error(y_train[y_train >= third_quantile], predictions[y_train >= third_quantile])

print(f'Error para valores bajos de energía: {low_values_error}')
print(f'Error para valores altos de energía: {high_values_error}')

# Convierte el problema de regresión en un problema de clasificación
y_train_class = np.where(y_train < third_quantile, 'baja', 'alta')

# Entrena un modelo de clasificación
clf = RandomForestClassifier()
clf.fit(X_train_scaled, y_train_class)

# Evalua el modelo de clasificación
predictions_class = clf.predict(X_train_scaled)
print(classification_report(y_train_class, predictions_class))